<a href="https://colab.research.google.com/github/qn19325/individualProject/blob/main/cnn_rnn_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [4]:
!pip install wandb --upgrade

In [5]:
import wandb 

wandb.login()

wandb: Currently logged in as: individual_project (use `wandb login --relogin` to force relogin)


True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
config = dict(
    epochs=1000,
    train_batch_size=8,
    test_batch_size=1,
    learning_rate=0.001,
    input_size = 128,
    hidden_size = 128,
    output_size = 1,
    sequence_length = 2,
    num_layers = 1,
    dataset="basic64x64",
    architecture="CNN/RNN")

In [8]:
def model_pipeline(hyperparameters):

    # tell wandb to get started
    with wandb.init(project="individual_project", config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      encoder, model, train_loader, test_loader, criterion, optimizer = make(config)
      print(model)

      # and use them to train the model
      main(encoder, model, train_loader, test_loader, criterion, optimizer, config)

    return model

In [9]:
def make(config):
    # Make the data
    dataset = get_data()
    train_loader = make_loader(dataset, batch_size=config.train_batch_size)
    test_loader = make_loader(dataset, batch_size=config.test_batch_size)

    # Make the CNN encoder
    encoder = Encoder().to(device)
    # Make the RNN model
    model = RNN(config).to(device)

    # Make the loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=config.learning_rate)
    
    return encoder, model, train_loader, test_loader, criterion, optimizer

In [10]:
def get_data():
    full_dataset = ImageDataLoader()
    
    return full_dataset

def make_loader(dataset, batch_size):
    loader = DataLoader(dataset=dataset, 
                        batch_size=batch_size, 
                        shuffle=True)
    return loader

In [11]:
class ImageDataLoader(Dataset):
    def __init__(self, dir_=None):
        self.data_df = pd.read_csv('gdrive/MyDrive/64x64.csv')
        self.dataset_len = len(self.data_df) # read the number of len of your csv files
    def __getitem__(self, idx):
        # load the next image
        f_name_t = self.data_df['Filename'][idx]
        f_name_tp1 = self.data_df['Filename'][idx+1]
        label = self.data_df['Label'][idx]
        label = label.astype(np.float32) 
        label = np.true_divide(label, 20)
        img_t = torchvision.io.read_image('gdrive/MyDrive/64x64/{}'.format(f_name_t))
        img_tp1 = torchvision.io.read_image('gdrive/MyDrive/64x64/{}'.format(f_name_tp1))
        img_t = img_t.float().div_(255.0)
        img_tp1 = img_tp1.float().div_(255.0)
        return img_t, img_tp1, label
    def __len__(self):
        return self.dataset_len - 1

In [7]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 8, 1, 1),
            nn.ReLU(),
            nn.Conv2d(8, 16, 1, 1),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc1 = nn.Linear(65536, 128)
    def forward(self, x):
        state = self.cnn(x)
        state = self.fc1(state)
        return state

In [8]:
class RNN(nn.Module):
    def __init__(self, config):
        super(RNN, self).__init__()
        self.input_size = config.get("input_size")
        self.num_layers = config.get("num_layers")
        self.hidden_size = config.get("hidden_size")
        self.output_size = config.get("output_size")
        self.rnn = nn.RNN(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device))
    def forward(self, x):
        self.batch_size = x.size(0)
        self.hidden = self.init_hidden()
        out, self.hidden = self.rnn(x, self.hidden)
        out = self.fc(out)
        return out

In [38]:
def main(encoder, model, train_loader, test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    example_ct = 0
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
      for _, (images1, images2, labels) in enumerate(train_loader):
        loss = train(images1, images2, labels, encoder, model, optimizer, criterion, epoch)
      
        example_ct += len(images1)
        batch_ct += 1

        # Report metrics every 25th batch
        if ((batch_ct + 1) % 25) == 0:
          train_log(loss, batch_ct, example_ct, epoch)

      if epoch % 5 == 0:
          test(test_loader, encoder, model, epoch)

      if epoch % 25 == 0:
        EPOCH = epoch
        PATH = "gdrive/MyDrive/models/cnn_rnn_" + str(epoch) + ".pt"
        LOSS = loss
        torch.save({
                    'epoch': EPOCH,
                    'model_state_dict': model.state_dict(),
                    'encoder_state_dict': encoder.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': LOSS,
                    }, PATH)

In [32]:
def train(images1, images2, labels, encoder, model, optimizer, criterion, epoch):
  model.train()

  images1, images2, labels = images1.to(device), images2.to(device), (labels.float()).to(device)
  
  # Forward pass ➡
  # pass to encoder
  output1 = encoder(images1)
  output2 = encoder(images2)
  # pass to RNN
  batch_size1 = len(output1)
  batch_size2 = len(output2)

  output1 = output1.reshape(batch_size1,1,-1)
  output2 = output2.reshape(batch_size2,1,-1)
  
  seq = torch.cat((output1, output2.detach()), dim=1)

  outputs = model(seq.to(device))
  loss = criterion(outputs[:,-1].squeeze(), labels.float())

  # Backward pass ⬅
  optimizer.zero_grad()
  loss.backward()

  # Step with optimizer
  optimizer.step()

  return loss

In [33]:
def train_log(loss, batch_ct, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Epoch: " + str(epoch) + f", Batch: " + str(batch_ct).zfill(4) + f", Example: " + str(example_ct).zfill(5) + f", Loss: {loss:.3f}")

In [40]:
def test(test_loader, encoder, model, epoch):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for images1, images2, labels in test_loader:
            images1, images2, labels = images1.to(device), images2.to(device), (labels.float()).to(device)
            output1 = encoder(images1)
            output2 = encoder(images2)
            # pass to RNN
            batch_size1 = len(output1)
            batch_size2 = len(output2)

            output1 = output1.reshape(batch_size1,1,-1)
            output2 = output2.reshape(batch_size2,1,-1)
            
            seq = torch.cat((output1, output2.detach()), dim=1)

            outputs = model(seq.to(device))
            # if epoch % 100 == 0:
            #   print(f"prediction:" + str(round(outputs[:,-1].item(), 5)) + f", labels:"+ str(round(labels.item(), 5)))
              
            predicted = round(outputs[:,-1].item(), 1)
            speed = round(labels.item(), 1)
            
            total += labels.size(0)
            if predicted == speed:
              correct += 1

        print(f"Accuracy of the model at epoch {epoch}: " + f"{100 * correct / total}%")
        
        wandb.log({"test_accuracy": correct / total})

In [41]:
model = model_pipeline(config)

RNN(
  (rnn): RNN(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 0, Batch: 0024, Example: 00192, Loss: 0.167
Epoch: 0, Batch: 0049, Example: 00392, Loss: 0.120
Epoch: 0, Batch: 0074, Example: 00592, Loss: 0.173
Epoch: 0, Batch: 0099, Example: 00792, Loss: 0.121
Epoch: 0, Batch: 0124, Example: 00992, Loss: 0.128
Accuracy of the model at epoch 0: 9.3812375249501%
Epoch: 1, Batch: 0149, Example: 01186, Loss: 0.109
Epoch: 1, Batch: 0174, Example: 01386, Loss: 0.159
Epoch: 1, Batch: 0199, Example: 01586, Loss: 0.093
Epoch: 1, Batch: 0224, Example: 01786, Loss: 0.044
Epoch: 1, Batch: 0249, Example: 01986, Loss: 0.049
Epoch: 2, Batch: 0274, Example: 02180, Loss: 0.137
Epoch: 2, Batch: 0299, Example: 02380, Loss: 0.116
Epoch: 2, Batch: 0324, Example: 02580, Loss: 0.099
Epoch: 2, Batch: 0349, Example: 02780, Loss: 0.080
Epoch: 2, Batch: 0374, Example: 02980, Loss: 0.106
Epoch: 3, Batch: 0399, Example: 03174, Loss: 0.122
Epoch: 3, Batch: 0424, Example: 03374, Loss: 0.073
Epoch: 3, Batch: 0449, Example: 03574, Loss: 0.055
Epoch: 3, Batch: 0474, Example:

wandb: ERROR Control-C detected -- Run data was not synced


KeyboardInterrupt: ignored

In [9]:
PATH = "gdrive/MyDrive/models/cnn_rnn_400.pt"

encoder = Encoder()
model = RNN(config)
optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=config.get("learning_rate"))

checkpoint = torch.load(PATH)
encoder.load_state_dict(checkpoint['encoder_state_dict'])
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [16]:
# def printgradnorm(self, grad_input, grad_output):
#     print('Inside ' + self.__class__.__name__ + ' backward')
#     print('Inside class:' + self.__class__.__name__)
#     print('')
#     print('grad_input: ', type(grad_input))
#     print('grad_input[0]: ', type(grad_input[0]))
#     print('grad_output: ', type(grad_output))
#     print('grad_output[0]: ', type(grad_output[0]))
#     print('')
#     print('grad_input size:', grad_input[0].size())
#     print('grad_output size:', grad_output[0].size())
#     print('grad_input norm:', grad_input[0].norm())
# model.rnn.register_forward_hook(printgradnorm)

# out = model(train_loader)
# err = loss_fn(out, target)
# err.backward()

NameError: ignored